In [1]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 36.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pulp as p

Problema:

Maximizar 950c + 1200h -> (Contribucion total)

sujeto a las restricciones:

1.4c + 2.8h <= 70

c + h <= 30

3c + h <= 65

c >= 0

h >= 0

In [26]:
# se instancia el modelo
modelo = p.LpProblem('Perfiles_Acero',p.LpMaximize)

In [27]:
# Aqui ya escribimos las restricciones >= 0 y h>= 0
C = p.LpVariable('Camione_Perfil_C',lowBound=0, cat=p.LpContinuous)
H = p.LpVariable('Camione_Perfil_H',lowBound=0, cat=p.LpContinuous)

In [28]:
# se declara la FO
modelo += 950 * C + 1200 * H, 'Función Objetivo'

In [29]:
# restricciones
modelo += 1.4 * C + 2.8 * H <= 70, 'Restriccion de capacidad de produccion'
modelo += C + H <= 30, 'Restriccion de capacidad de carga'
modelo += 3*C + H <= 65, 'Restriccion de Límite de Recursos'

In [30]:
modelo


Perfiles_Acero:
MAXIMIZE
950*Camione_Perfil_C + 1200*Camione_Perfil_H + 0
SUBJECT TO
Restriccion_de_capacidad_de_produccion: 1.4 Camione_Perfil_C
 + 2.8 Camione_Perfil_H <= 70

Restriccion_de_capacidad_de_carga: Camione_Perfil_C + Camione_Perfil_H <= 30

Restriccion_de_Límite_de_Recursos: 3 Camione_Perfil_C + Camione_Perfil_H <= 65

VARIABLES
Camione_Perfil_C Continuous
Camione_Perfil_H Continuous

In [31]:
# resolvemos
modelo.solve()

1

Las variables de decición serian: 10 perfiles tipo C y 20 perfiles tipo 20.

La utilidad máxima sería de $ 33,500

In [32]:
print(f'Número de perfiles tipo C {C.varValue}')
print(f'Número de perfiles tipo h {H.varValue}')
print(f'Utilidad {p.value(modelo.objective)}')

Número de perfiles tipo C 10.0
Número de perfiles tipo h 20.0
Utilidad 33500.0


**Análisis de sensibilidad**



Holgura positiva indica que existe un excedente de recursos y la restricción s considera inactiva. Este recurso no se está consumuento en su totalidad, por lo tanto no limita el problema de programación lineal.

Frente a holgura negativa , una restricción se considera activa pues corresponde a un recurso que se agota. Constituye una limitación real para el problema


In [33]:
sensibilidad = [
    {
        'Restricción':i, 'Precio Sombra':j.pi, 'Holgura':j.slack
    }    for i,j in modelo.constraints.items()
]

sensibilidad = pd.DataFrame(sensibilidad)
display(sensibilidad)

,Restricción,Precio Sombra,Holgura
0,Restriccion_de_capacidad_de_produccion,178.57143,-0.0
1,Restriccion_de_capacidad_de_carga,700.00000,-0.0
2,Restriccion_de_Límite_de_Recursos,-0.00000,15.0


Reto: resolve el problema planteado en la sección 1:

Zmax = 40 X + 30 Y

restricciones

x <= 40

y >= 0

x + y  <= 80

2 x + y <= 100

In [54]:
# se instancia el modelo
modelo = p.LpProblem('Unidades_de_producto',p.LpMaximize)
# Variables de decision
# Aqui ya escribimos las restricciones >= 0 y h>= 0
x = p.LpVariable('unidades_producto_x',lowBound=0, upBound=40, cat=p.LpContinuous)
y = p.LpVariable('unidades_producto_y',lowBound=0, cat=p.LpContinuous)

# restricciones
modelo += x + y <= 80, 'Restriccion de capacidad de horas proceso A'
modelo += 2*x + y <= 100, 'Restriccion de capacidad de horas proceso B'

# se declara la FO
modelo += 40 * x + 30 * y, 'Función Objetivo'

modelo

Unidades_de_producto:
MAXIMIZE
40*unidades_producto_x + 30*unidades_producto_y + 0
SUBJECT TO
Restriccion_de_capacidad_de_horas_proceso_A: unidades_producto_x
 + unidades_producto_y <= 80

Restriccion_de_capacidad_de_horas_proceso_B: 2 unidades_producto_x
 + unidades_producto_y <= 100

VARIABLES
unidades_producto_x <= 40 Continuous
unidades_producto_y Continuous

In [55]:
modelo.solve()

1

In [56]:
print(f'Número de unidades X {x.varValue}')
print(f'Número de unidades Y {y.varValue}')
print(f'Utilidad maxima {p.value(modelo.objective)}')

Número de unidades X 20.0
Número de unidades Y 60.0
Utilidad maxima 2600.0


In [57]:
sensibilidad = [
    {
        'Restricción':i, 'Precio Sombra':j.pi, 'Holgura':j.slack
    }    for i,j in modelo.constraints.items()
]

sensibilidad = pd.DataFrame(sensibilidad)
display(sensibilidad)

,Restricción,Precio Sombra,Holgura
0,Restriccion_de_capacidad_de_horas_proceso_A,20.0,-0.0
1,Restriccion_de_capacidad_de_horas_proceso_B,10.0,-0.0
